# DATA SETS

* FIFA 21: Archivo *=>* [ FIFA-21 Complete.csv ](https://www.kaggle.com/datasets/aayushmishra1512/fifa-2021-complete-player-data?resource=download)

El data set original cuenta con 9 columnas: "player_id", "name", "nationality", "position", "overall", "age", "hits", "potential" y "team". En ese orden.

El objetivo es obtener un archivo csv, con las columnas: "Team", "Nationality", "Position", "Age", "Potential" y "Name". En ese orde. Sin embargo, antes, se debe hacer algunas conversiones de datos para poder ser utilizados.

* Traducción de las posiciones de los jugadores, del ingles al español. "DK" debe pasar a ser "Portero", por ejemplo.

Para ello, se utiliza la siguiente función: 

In [31]:
#imports necesarios
import csv
import json
from os import path #utilizado para obtener la rutas de archivos

In [32]:
def traduciones():
    """
    :return: las traducciones de las posiciones de los jugadores
    """
    # Se almacena, en un diccionario, la traduccion (valor) 
    # de la posicion (clave)
    try:
        ruta_completa = path.join(path.realpath('..'), "csv_originales" ,"fifa_pos_ENG_ESP.json")
        with open(ruta_completa, 'r', encoding="utf-8") as traduccion:
            return json.load(traduccion)
    except FileNotFoundError:
        return ("No encontrado")

* Reemplazar el valor de 'potential', que contiene un valor numerico fijo, a su equivalente en palabras.

Para ello, se utiliza la siguiente función: 

In [33]:
def conversion_del_potencial(potencial):
    """
    :param potencial: el valor de la columna 'Potential'
    :return: el equivalente a potencial en string
    """
    if(potencial < "60"):
        potencial = "Regular"
    elif(potencial > "60" and potencial <= "79"):
        potencial = "Bueno"
    elif(potencial > "79" and potencial <= "89"):
        potencial = "Muy bueno"
    else:
        potencial = "Sobresaliente"
    return potencial

* Las funciones anteriores, son implementadas en otra función que recibirá el iterador del csv y aplicará, a cada fila, las anteriores funciones. Retornando el resultado en una lista.

In [34]:
def procesar_filas(csvreader):
    """
    :param csvreader: el contenido del archivo en una lista
    :return: una lista filtrada segun los criterios adoptados
    """
    lista_filtrada = []
    trad = traduciones()
    for ln in csvreader:
        # conversion de numero a strig segun el potencial
        ln[7] = conversion_del_potencial(ln[7])
        # traduccion de las posiciones
        # se aplica un split ya que, un jugador, puede tener varias posiciones 
        ln[3] = '|'.join(
            list(map(lambda pos: trad[pos], ln[3].split('|'))))
        lista_filtrada.append(ln)

    return lista_filtrada

* Invocación de todo lo anterior, para luego generar un nuevo archivo csv que cumpla con las condiciones planteadas para ser utilizado.

In [41]:
try:
    ruta_completa = path.join(path.realpath('..'), "csv_originales" ,"FIFA-21 Complete.csv")
    with open(ruta_completa, 'r', encoding="utf-8") as fifa:
        csvreader = csv.reader(fifa, delimiter=';')
        next(csvreader)  # omito el encabezado
        filas = procesar_filas(csvreader)
        # reacomodo las columnas, por fila
        lista_filtrada = list(map(lambda fila: [fila[8], fila[2], fila[3], fila[5], fila[7], fila[1]][0:6], filas))

    ruta_completa = path.join(path.realpath('..'), "dataset_fifa.csv")

    with open(ruta_completa, 'w', encoding="utf-8", newline='') as fifa:
        writer = csv.writer(fifa)
        # Escribo el nuevo encabezado
        writer.writerow(["Team", "Nationality", "Position", "Age", "Potential", "Name"])
        writer.writerows(lista_filtrada)
except FileNotFoundError:
    print("Archivo csv 'FIFA-21 complete.csv' no encontrado")